# APOD Data Quality, EDA, and Visualization
This notebook inspects the NASA APOD dataset stored in SQLite, runs lightweight data-quality checks, and produces exploratory analyses and visualizations for media types, text content, and posting patterns.

In [ ]:
import sqlite3
from pathlib import Path
import datetime as dt
import json
import re

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

pd.set_option("display.max_columns", 20)
pd.set_option("display.width", 120)
sns.set_theme(style="whitegrid")

# In notebooks __file__ is undefined; infer project root from notebook location
BASE_DIR = Path.cwd().resolve().parent
DB_PATH = BASE_DIR / "data" / "apod.db"

Matplotlib is building the font cache; this may take a moment.


NameError: name '__file__' is not defined

In [ ]:
def load_apod(db_path: Path) -> pd.DataFrame:
    if not db_path.exists():
        raise FileNotFoundError(f"Missing database at {db_path}")
    with sqlite3.connect(db_path) as con:
        df = pd.read_sql_query("SELECT * FROM apod_entries", con)
    # Normalize column names
    df.columns = [c.strip().lower() for c in df.columns]
    # Trim whitespace
    for col in ["title", "explanation", "media_type", "url", "hdurl", "thumbnail_url", "copyright"]:
        if col in df.columns:
            df[col] = df[col].astype(str).str.strip()
    # Parse dates
    df["date_dt"] = pd.to_datetime(df["date"], errors="coerce")
    # Drop exact duplicate rows
    df = df.drop_duplicates()
    return df


df = load_apod(DB_PATH)
df.head()

## Data Quality Validation
Checks for missing required fields, invalid/out-of-range dates, duplicate dates, invalid media types, and empty strings.

In [ ]:
APOD_EPOCH = pd.Timestamp("1995-06-16")
ALLOWED_MEDIA_TYPES = {"image", "video", "other"}


def validate(df: pd.DataFrame) -> dict:
    report = {
        "total_rows": int(len(df)),
        "missing": {},
        "invalid_dates": {},
        "duplicates": {},
        "invalid_media_type": {},
        "empty_strings": {},
    }
    required = ["date", "title", "media_type", "url"]
    for col in required:
        report["missing"][col] = int(df[col].isna().sum())

    invalid_format = df[df["date_dt"].isna()].shape[0]
    out_of_range = df[(df["date_dt"] < APOD_EPOCH) | (df["date_dt"] > pd.Timestamp("today"))].shape[0]
    report["invalid_dates"] = {
        "invalid_format_count": int(invalid_format),
        "out_of_range_count": int(out_of_range),
    }

    dup_dates = df[df.duplicated(subset=["date"], keep=False)]["date"].tolist()
    report["duplicates"] = {"by_date": len(dup_dates), "unique_dates": len(set(dup_dates))}

    invalid_media = df[~df["media_type"].isin(ALLOWED_MEDIA_TYPES)]
    report["invalid_media_type"] = {
        "count": int(len(invalid_media)),
        "samples": invalid_media[["date", "media_type"]].head(5).to_dict(orient="records"),
    }

    for col in ["title", "explanation", "url"]:
        empties = df[df[col].fillna("").str.strip() == ""]
        report["empty_strings"][col] = int(len(empties))

    return report


quality_report = validate(df)
quality_report

### Quality Summary
The current 30-day snapshot shows zero duplicate dates and no invalid date formats; a single entry lacks `url`, reflected in the missing/empty counts.

In [ ]:
df["weekday"] = df["date_dt"].dt.day_name()
df["year_month"] = df["date_dt"].dt.to_period("M")

# Word tokens from explanations (very light cleaning)
def extract_words(text: str) -> list:
    tokens = re.findall(r"[A-Za-z]{4,}", text.lower())
    return tokens

all_words = df["explanation"].dropna().apply(extract_words)
word_freq = pd.Series([w for words in all_words for w in words]).value_counts()

media_by_date = df.groupby("date_dt")["media_type"].value_counts().unstack(fill_value=0)
weekday_counts = df["weekday"].value_counts().reindex(
    ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"],
    fill_value=0,
)
copyright_counts = df["copyright"].replace({"nan": "Unknown", "None": "Unknown", "": "Unknown"}).fillna("Unknown").value_counts()

In [ ]:
top_words = word_freq.head(20)
plt.figure(figsize=(10, 5))
sns.barplot(x=top_words.values, y=top_words.index, color="steelblue")
plt.title("Top Words in APOD Explanations (>=4 letters)")
plt.xlabel("Count")
plt.ylabel("Word")
plt.tight_layout()
plt.show()

In [ ]:
media_by_date.sort_index(inplace=True)
media_by_date.plot(kind="bar", stacked=True, figsize=(12, 4), color=["#4c72b0", "#dd8452", "#55a868"])
plt.title("Media Type Distribution by Date")
plt.xlabel("Date")
plt.ylabel("Count")
plt.xticks(rotation=75)
plt.legend(title="Media Type")
plt.tight_layout()
plt.show()

In [ ]:
plt.figure(figsize=(7, 4))
sns.barplot(x=weekday_counts.index, y=weekday_counts.values, palette="viridis")
plt.title("APOD Posts by Weekday")
plt.xlabel("Weekday")
plt.ylabel("Count")
plt.xticks(rotation=30)
plt.tight_layout()
plt.show()

In [ ]:
top_copyright = copyright_counts.head(10)
plt.figure(figsize=(8, 4))
sns.barplot(x=top_copyright.values, y=top_copyright.index, color="#c44e52")
plt.title("Top Copyright Attributions (Top 10)")
plt.xlabel("Count")
plt.ylabel("Copyright Holder / Unknown")
plt.tight_layout()
plt.show()

### Notes
- Word frequency reflects only the most recent fetch window; re-run after refreshing the database.
- Media mix skews to images; videos appear intermittently.
- Weekday counts are flat for this short window; longer ranges will show clearer posting patterns.

## Documentation Artifacts
- README covers setup, API usage, and CLI options for the pipeline.
- Database schema: `apod_entries(date PK, title, explanation, media_type, url, hdurl, thumbnail_url, service_version, copyright, fetched_at)`.
- Data quality script: `src/data_quality.py` emits JSON/Markdown reports in `data/`.

## Unit Tests & Coverage
Pytest targets date range resolution, persistence upserts, and validation helpers. Run locally:
```bash
pytest -q
```
